# How to run this notebook
`bazel build  lib/GaussianFilters:jupyter && ./bazel-bin/lib/GaussianFilters/jupyter`

# The $\alpha - \beta - \gamma$ filter


## The $\alpha$ filter
- We want to estimate the state of a static system, ie a system that does not change its state over a reasonable period.
- We also assume that our measurements have a random error, but are not biased.

Such example is measuring the weight of an object. Intuitively, the easiest way to do this is to compute the average of the measurements. The problem is that it is time and memory consuming. We are looking for an iterative method.

We use the following notation:
- $x$ = true value
- $z_n$ = measurement at time $n$
- $\hat{x}_{i,j}$ = estimate of state $i$ after taking measurement $j$

Then we can write the average as $$\hat{x}_{n, n} = \frac{1}{n} \sum^n_{i=1} z_i$$.

We can rewrite this as $$\hat{x}_{n,n} = \hat{x}_{n, n-1} + \frac{1}{n} (z_n - \hat{x}_{n,n-1})$$. 
In the context of a Kalman filter, it is the **State Update Equation** and can be more generaly writen as $$\hat{x}_{n,n} = \hat{x}_{n, n-1} + \alpha_n (z_n - \hat{x}_{n,n-1})$$. 

In practice it means that the estimate of the current state is equal to the predicted value of the previous prediction plus a factor times the different between the current measurement and the previous estimate (= new information). 
The factor $\alpha$ is called Kalmar gain, also written $K_n$. In our case we see that it decreases with time, ie the new information decreases.

In [1]:
class Model:
    @staticmethod
    def compute_alpha(n):
        return 1 / n
    
    @staticmethod
    def state_update_equation(prev_state, factor, measurement):
        innovation = measurement - prev_state
        return prev_state + factor * innovation

    
model = Model()
# Step 1: Measure
measurements = [1000, 1030, 989, 1017, 1009, 1013, 979, 1008, 1042, 1012, 1011]
n = 0
x_n_n = measurements[n]
print(f"State {n} : x_n_n = {x_n_n}")

# Step 2: Update and predict
for n in range(1, len(measurements)):
    zn = measurements[n]
    Kn = model.compute_alpha(n)
    x_n_n_prev = x_n_n
    x_n_n = model.state_update_equation(x_n_n_prev, Kn, zn)
    print(f"State {n} : x_n_n = {x_n_n}")


State 0 : x_n_n = 1000
State 1 : x_n_n = 1030.0
State 2 : x_n_n = 1009.5
State 3 : x_n_n = 1012.0
State 4 : x_n_n = 1011.25
State 5 : x_n_n = 1011.6
State 6 : x_n_n = 1006.1666666666667
State 7 : x_n_n = 1006.4285714285714
State 8 : x_n_n = 1010.875
State 9 : x_n_n = 1011.0
State 10 : x_n_n = 1011.0


## $\alpha - \beta$ filter
$\alpha - \beta$ filters are used when a system is adequately approximated by a model having two internal states, where the first state is obtained by integrating the value of the second state over time.
- The $\alpha$ constant corrects the position (ie the first state) using the innovation.
- The $\beta$ constant corrects the velocity (ie the second state) using the innovation.

We will estimate the state of a dynamic system, in that case the speed of an object in one dimension.
Let's define:
- $x_n$ = the position of the object at time $n$.
- $\Delta t$ = the update interval

From this we can compute the velocity of the object $$\dot x = v = \frac{dx}{dt}$$.

We can now describe the *dynamic model* of the problem, in our case motion with a constant velocity:
- $x_{n+1} = x_n + \Delta t \dot x_n$ : the next position is equal to the current plus the speed times interval time.
- $\dot x_{n+1} = \dot x_n$ : since we have constant velocity, it remains unchanged accross steps.

This system of equations is called **State Extrapolation Equation** (or Transition Equation / Prediction Equation). It is one of the five Kalmar filter equations and is used to copmute the next state from the current state. In the previous example, it was simply $x_{n+1} = x_n$.

As we saw, since the system is dynamic, we have two **State Update Equations** this time:
- $\hat x_{n,n} = \hat x_{n,n-1} + \alpha (z_n - \hat x_{n,n-1}$ for the position
- $\hat{\dot x}_{n,n} = \hat{\dot x}_{n,n-1} + \beta (\frac{z_n - \hat x_{n,n-1}}{\Delta t})$ for the velocity

How are $\alpha$ and $\beta$ used here ?
- Here $\alpha$ is fixed, and depends on the precision of the sensor. If it has a high precision, we can have confidence in the measurements and use a high $\alpha$. If $\alpha = 1$, then the estimated position is the measured position, ie $\hat x_{n,n} = z_n$. If $\alpha = 0$ then the measurement is discarded, ie $\hat x_{n,n} = \hat x_{n,n-1}$.
- $\beta$ also depends on the precision of the sensor. If the precision is high, we use a high $\beta$, because we assume that a change in the observed velocity is likely due to a real change in velocity (ie we trust the measurement, and we should use it to update the velocity). In the oposite case, a low $\beta$ gives a lower contribution to the measurement in updating the velocity.

In [2]:
class Model:
    Δt = 5
    
    @staticmethod
    def compute_alpha(n):
        return 0.2
    
    @staticmethod
    def compute_beta(n):
        return 0.1
    
    @staticmethod
    def state_extrapolation_equation():
        
    
    @staticmethod
    def state_update_equation(prev_state, factor, measurement):
        innovation = measurement - prev_state
        return prev_state + factor * innovation

    
model = Model()
# Step 1: Measure
measurements_position = [30000]
measurements_velocity = [40]
n = 0
x_n_n = measurements[n]
print(f"State {n} : x_n_n = {x_n_n}")

# Step 2: Update and predict
for n in range(1, len(measurements)):
    zn = measurements[n]
    Kn = model.compute_Kn(n)
    x_n_n_prev = x_n_n
    x_n_n = model.state_update_equation(x_n_n_prev, Kn, zn)
    print(f"State {n} : x_n_n = {x_n_n}")

State 0 : x_n_n = 1000


AttributeError: 'Model' object has no attribute 'compute_Kn'